In [1]:
#create imports for bassic pytorch lstm
import os
import torch
import numpy as np
import pandas as pd

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
batch_size = 10
# FashionMNIST is downloaded from the internet if not already present in './data'
training_data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transforms.ToTensor())

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [3]:
import matplotlib.pyplot as plt

# FashionMNIST class labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Get a small sample of data (first 5 items)
dataiter = iter(train_dataloader)
images, labels = next(dataiter)

# Show basic dataset info
print("=== FashionMNIST Dataset Head ===")
print(f"Training dataset size: {len(training_data)}")
print(f"Test dataset size: {len(test_data)}")
print(f"Number of classes: {len(class_names)}")
print(f"Batch size: {batch_size}")
print(f"Image shape: {images[0].shape}")
print()

# Display first 5 samples as a simple overview
print("First 5 samples in this batch:")
for i in range(5):
    label_idx = labels[i].item()
    img_shape = images[i].shape
    pixel_range = f"[{images[i].min():.3f}, {images[i].max():.3f}]"
    print(f"Sample {i+1}: {class_names[label_idx]:<12} | Shape: {img_shape} | Pixel range: {pixel_range}")

# Show just the first 3 images in a simple row
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
fig.suptitle('First 3 Samples from FashionMNIST', fontsize=14)

for i in range(3):
    img = images[i].squeeze().numpy()
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f'{class_names[labels[i]]}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

=== FashionMNIST Dataset Head ===
Training dataset size: 60000
Test dataset size: 10000
Number of classes: 10
Batch size: 10
Image shape: torch.Size([1, 28, 28])

First 5 samples in this batch:
Sample 1: Ankle boot   | Shape: torch.Size([1, 28, 28]) | Pixel range: [0.000, 1.000]
Sample 2: T-shirt/top  | Shape: torch.Size([1, 28, 28]) | Pixel range: [0.000, 1.000]
Sample 3: T-shirt/top  | Shape: torch.Size([1, 28, 28]) | Pixel range: [0.000, 1.000]
Sample 4: Dress        | Shape: torch.Size([1, 28, 28]) | Pixel range: [0.000, 1.000]
Sample 5: T-shirt/top  | Shape: torch.Size([1, 28, 28]) | Pixel range: [0.000, 1.000]


: 

In [ ]:
# define hyperparameters
sequence_len = 28
input_len = 28
hidden_size = 128
num_layers = 2
num_classes = 10
num_epochs = 5
learning_rate = 0.01


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_len, hidden_size, num_classes, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        hidden_state = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # (num_layers, batch_size, hidden_size)
        cell_state = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # (num_layers, batch_size, hidden_size)
        out, _ = self.lstm(x, (hidden_state, cell_state)) # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = self.output_layer(out[:, -1, :])  
        return out
    






In [ ]:
model = LSTMModel(input_len, hidden_size, num_classes, num_layers)
print(model)


In [ ]:
loss_function = nn.CrossEntropyLoss()
sgd = optim.SGD(model.parameters(), lr=learning_rate)
adam = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = sgd

In [ ]:
def train(num_epochs, model, train_dataloader, loss_function):
    total_step = len(train_dataloader)

    for epoch in range(num_epochs):
        for batch, (image, labels) in enumerate(train_dataloader):
            images = image.reshape(-1, sequence_len, input_len)

            outputs = model(images)
            loss = loss_function(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch+1) % batch_size == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch+1}/{total_step}], Loss: {loss.item():.4f}")

In [ ]:
train(num_epochs, model, train_dataloader, loss_function)

In [ ]:
test_images, test_labels = next(iter(test_dataloader))
test_labels

In [ ]:
test_output = model(test_images.view(-1,28,28))

In [ ]:
predicted = torch.max(test_output, 1)[1]
predicted

In [ ]:
correct = [1 for i in range(batch_size) if predicted[i] == test_labels[i]]
percentage_correct = sum(correct)/batch_size 

In [ ]:
percentage_correct